# Section 1: Data Exploration（データ探索）

## 🎯 このセクションの目標
- Snowflake Notebookの基本操作を習得する
- デモデータの構造と内容を理解する
- チャーンの定義を決め、ラベルを作成する

## 📊 シナリオ: Snow Retail社のチャーン予測

架空の小売会社「Snow Retail」の顧客データを分析します。

**ビジネス課題:**
- 顧客のチャーン（離反）を予測したい
- 離反しそうな顧客に事前にアプローチしたい

---
## 1.1 環境設定

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F

session = get_active_session()
session.use_database("MLOPS_HOL_DB")
session.use_schema("PREP_DATA")
session.use_warehouse("MLOPS_HOL_PYTHON_WH")

print("✅ 環境設定完了")

---
## 1.2 顧客データの確認

In [ ]:
customers = session.table("CUSTOMERS")
print(f"顧客数: {customers.count():,}")
customers.show(10)

In [ ]:
# セグメント別顧客数
print("=== セグメント別顧客数 ===")
customers.group_by("SEGMENT").count().order_by("SEGMENT").show()

---
## 1.3 注文データの確認

In [ ]:
orders = session.table("ORDERS")
print(f"注文数: {orders.count():,}")
orders.show(10)

In [ ]:
# 期間別注文数
print("=== 期間別注文数 ===")
orders.with_column(
    "PERIOD",
    F.when(F.col("ORDER_DATE") < "2024-01-01", "2023年")
     .when(F.col("ORDER_DATE") <= "2024-06-30", "2024年前半")
     .otherwise("2024年後半")
).group_by("PERIOD").count().order_by("PERIOD").show()

---
## 1.4 チャーンの定義とラベル作成 🎯

このステップでチャーンラベルを作成します。

### チャーンの定義

> **2024年前半（1-6月）に注文があったが、2024年後半（7-12月）に注文がなかった顧客**

| 期間 | 用途 |
|------|------|
| 2024年前半（1-6月） | 特徴量計算（この期間の行動を分析） |
| 2024年後半（7-12月） | チャーン判定（この期間に注文なし＝離反） |

### なぜこの定義？
- **観測可能**: 2024年前半のデータから特徴量を計算
- **予測可能**: 後半に離反するかを予測
- **実用的**: 「最近注文がない = 離反」という直感的な定義

In [ ]:
# チャーンラベルを注文データから計算する

# Step 1: 2024年前半に注文がある顧客
h1_customers = orders.filter(
    (F.col("ORDER_DATE") >= "2024-01-01") & 
    (F.col("ORDER_DATE") <= "2024-06-30")
).select("CUSTOMER_ID").distinct()

print(f"2024年前半に注文した顧客数: {h1_customers.count():,}")

# Step 2: 2024年後半に注文がある顧客
h2_customers = orders.filter(
    F.col("ORDER_DATE") >= "2024-07-01"
).select("CUSTOMER_ID").distinct()

print(f"2024年後半に注文した顧客数: {h2_customers.count():,}")

In [ ]:
# Step 3: チャーンラベルを作成
# チャーン = 前半に注文あり、後半に注文なし

# 後半に注文がなかった顧客 = チャーン (IS_CHURNED = 1)
churned = h1_customers.join(
    h2_customers, 
    on="CUSTOMER_ID", 
    how="left_anti"  # h2_customersにないものだけ
).with_column("IS_CHURNED", F.lit(1))

# 後半にも注文があった顧客 = 非チャーン (IS_CHURNED = 0)
not_churned = h1_customers.join(
    h2_customers, 
    on="CUSTOMER_ID", 
    how="inner"  # 両方にあるもの
).with_column("IS_CHURNED", F.lit(0))

# 結合
churn_labels = churned.union_all(not_churned)

print(f"\n=== チャーンラベル生成完了 ===")
print(f"総顧客数: {churn_labels.count():,}")

In [ ]:
# チャーン分布を確認
print("=== チャーン分布 ===")
churn_labels.group_by("IS_CHURNED").count().show()

# チャーン率を計算
churn_rate = churn_labels.agg(F.avg("IS_CHURNED")).collect()[0][0] * 100
print(f"\n📊 チャーン率: {churn_rate:.1f}%")
print("\n💡 データからチャーンラベルを作成しました")

In [ ]:
# チャーンラベルをテーブルに保存（後続のNotebookで使用）
churn_labels.write.save_as_table(
    "MLOPS_HOL_DB.PREP_DATA.CHURN_LABELS",
    mode="overwrite"
)

print("✅ チャーンラベルを CHURN_LABELS テーブルに保存しました")

---
## 1.5 セグメント別チャーン率

In [ ]:
# 顧客とチャーンラベルを結合
customer_churn = customers.join(churn_labels, on="CUSTOMER_ID")

# セグメント別チャーン率
print("=== セグメント別チャーン率 ===")
segment_churn = customer_churn.group_by("SEGMENT").agg(
    F.count("*").alias("COUNT"),
    F.sum("IS_CHURNED").alias("CHURNED"),
    F.round(F.sum("IS_CHURNED") / F.count("*") * 100, 1).alias("CHURN_RATE_PCT")
).order_by(F.col("CHURN_RATE_PCT").desc())
segment_churn.show()

---
## 1.6 注文パターンとチャーンの関係（2024年前半データ）

In [ ]:
# 2024年前半（特徴量計算期間）の注文集計
from datetime import date
OBSERVATION_DATE = date(2024, 6, 30)

orders_h1 = orders.filter(F.col("ORDER_DATE") <= F.lit(OBSERVATION_DATE))

customer_stats = orders_h1.group_by("CUSTOMER_ID").agg(
    F.datediff("day", F.max("ORDER_DATE"), F.lit(OBSERVATION_DATE)).alias("DAYS_SINCE_LAST"),
    F.count("*").alias("ORDER_COUNT"),
    F.round(F.avg("ORDER_AMOUNT"), 2).alias("AVG_AMOUNT")
)

# チャーンラベルと結合
stats_with_churn = customer_stats.join(churn_labels, on="CUSTOMER_ID")

# チャーン vs 非チャーンの比較
print("=== チャーン vs 非チャーン（2024年前半データ）===")
comparison = stats_with_churn.group_by("IS_CHURNED").agg(
    F.round(F.avg("DAYS_SINCE_LAST"), 1).alias("AVG_DAYS_LAST_ORDER"),
    F.round(F.avg("ORDER_COUNT"), 1).alias("AVG_ORDERS"),
    F.round(F.avg("AVG_AMOUNT"), 2).alias("AVG_AMOUNT")
)
comparison.show()

### 💡 発見

上記の結果から、チャーン顧客の特徴が見えてきます：

| 特徴 | チャーン顧客 | アクティブ顧客 |
|------|------------|---------------|
| 最終注文からの日数 | **長い** | 短い |
| 注文回数 | **少ない** | 多い |

これらが重要な特徴量になります。

---
## ✅ Section 1 完了！

### 学んだこと
- Snowflake Notebookの基本操作
- デモデータの構造（顧客、注文）
- **チャーンの定義を決め、ラベルを作成** 
- チャーン顧客の特徴分析

### チャーンの定義

> **2024年前半（1-6月）に注文があったが、2024年後半（7-12月）に注文がなかった顧客**

### 作成したもの
- **CHURN_LABELS テーブル**: 作成したチャーンラベル

### 次のステップ
**Section 2: Feature Store** では、2024年前半のデータから特徴量を計算し、Feature Storeに登録します。

➡️ `02_FEATURE_STORE` Notebookに進んでください